In [ ]:
from PySDM_examples.Shipway_and_Hill_2012 import Simulation, Settings
from PySDM.physics import si
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

In [ ]:
settings = Settings(n_sd_per_gridbox=2048, precip=False, p0=1007 * si.hPa, particles_per_volume_STP=1e10, z_max=2000, dz=2000/256, t_max=1000, dt=0.1, rho_times_w_1=1 * si.m / si.s * si.kg / si.m**3, kappa=0.1)
simulation = Simulation(settings)

In [ ]:
results = simulation.run()

In [ ]:
products = results.products

In [ ]:
products.keys()

In [ ]:
plt.ylim((100,2000))
plt.pcolormesh(products["t"], products["z"], products["nc"])
plt.colorbar()

In [ ]:
plt.plot(products["z"], products["nc"][:,-1])

In [ ]:
PySDM_ds = xr.Dataset(
    data_vars={
        "N_aer" : (["t", "zc"], products["na"].T),
        "N_liq" : (["t", "zc"], products["nc"].T),
        "q_v" : (["t", "zc"], products["qv"].T / (products["qv"].T + 1)),
        "q_liq" : (["t", "zc"], (products["qc"].T / 1000) / (products["qc"].T / 1000 + 1)),
    },
    coords={
        "t" : products["t"],
        "zc" : products["z"],
    }
)
PySDM_ds

In [ ]:
PySDM_ds.to_netcdf("kinematic_simulations/PySDM_output_2.nc")

In [ ]:
plt.ylim(0, 2000)
plt.xlabel("t (s)")
plt.ylabel("z (m)")
plt.title("PySDM N_liq")
plt.pcolormesh(PySDM_ds["t"], PySDM_ds["zc"], (PySDM_ds["N_liq"]).T)
plt.colorbar()

In [ ]:
KiD_ARG_ds = xr.open_dataset("kinematic_simulations/KiD_output_ARG_2.nc", group="profiles")
KiD_ARG_ds

In [ ]:
ARG_diff_ds = (KiD_ARG_ds.interp(PySDM_ds.coords) - PySDM_ds)

In [ ]:
KiD_emulated_ds = xr.open_dataset("kinematic_simulations/KiD_output_emulated_2.nc", group="profiles")
KiD_emulated_ds

In [ ]:
emulated_diff_ds = (KiD_emulated_ds.interp(PySDM_ds.coords) - PySDM_ds)

In [ ]:
plt.ylim(0, 2000)
plt.xlabel("t (s)")
plt.ylabel("z (m)")
plt.title("ARG Scheme N_liq")
plt.pcolormesh(KiD_ARG_ds["t"], KiD_ARG_ds["zc"], KiD_ARG_ds["N_liq"].T)
plt.colorbar()

In [ ]:
plt.ylim(0, 2000)
plt.xlabel("t (s)")
plt.ylabel("z (m)")
plt.title("ARG Scheme N_liq Deviation from PySDM")
plt.pcolormesh(ARG_diff_ds["t"], ARG_diff_ds["zc"], ARG_diff_ds["N_liq"].T, cmap="coolwarm")
plt.colorbar()

In [ ]:
plt.ylim(0, 2000)
plt.xlabel("t (s)")
plt.ylabel("z (m)")
plt.title("Emulated N_liq")
plt.pcolormesh(KiD_emulated_ds["t"], KiD_emulated_ds["zc"], KiD_emulated_ds["N_liq"].T)
plt.colorbar()

In [ ]:
plt.ylim(0, 2000)
plt.xlabel("t (s)")
plt.ylabel("z (m)")
plt.title("Emulated N_liq Deviation from PySDM")
plt.pcolormesh(emulated_diff_ds["t"], emulated_diff_ds["zc"], emulated_diff_ds["N_liq"].T, cmap="coolwarm")
plt.colorbar()

In [ ]:
diff_diff_ds = np.abs(emulated_diff_ds) - np.abs(ARG_diff_ds)

In [ ]:
plt.ylim(0, 2000)
plt.xlabel("t (s)")
plt.ylabel("z (m)")
plt.title("Difference of Absolute Deviations (Emulated - ARG)")
plt.pcolormesh(diff_diff_ds["t"], diff_diff_ds["zc"], diff_diff_ds["N_liq"].T, cmap="RdYlGn_r")
plt.colorbar()

In [ ]:
plt.xlabel("N_liq (m^-3)")
plt.ylabel("z (m)")
plt.ylim(0, 2000)
plt.plot(PySDM_ds["q_liq"][-1,:], PySDM_ds["zc"], label="PySDM")
plt.plot(KiD_ARG_ds["q_liq"][-1,:], KiD_ARG_ds["zc"], label="ARG")
plt.plot(KiD_emulated_ds["q_liq"][-1,:], KiD_emulated_ds["zc"], ls="--", label="Emulated")
plt.legend();

In [ ]:
plt.xlabel("N_aer (m^-3)")
plt.ylabel("z (m)")
plt.ylim(0, 2000)
plt.plot(PySDM_ds["N_aer"][-1,:], PySDM_ds["zc"], label="PySDM")
plt.plot(KiD_ARG_ds["N_aer"][-1,:], KiD_ARG_ds["zc"], label="ARG")
plt.plot(KiD_emulated_ds["N_aer"][-1,:], KiD_emulated_ds["zc"], ls="--", label="Emulated")
plt.legend();